In [1]:
from skrub import tabular_learner
import os
import numpy as np

In [2]:
os.chdir("../data")
import pandas as pd
df = pd.read_csv("quejas-clientes.csv", low_memory=False)
df.drop(columns=["Unnamed: 0"], inplace=True)

In [3]:
df.fillna("Unknown or not specified", inplace= True)

C:\Users\unaif\AppData\Local\Temp\ipykernel_14060\3163457211.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown or not specified' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown or not specified", inplace= True)


In [4]:
df=df[df["Consumer disputed?"]!="Unknown or not specified"]
df

,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",Unknown or not specified,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
6,1288848,Consumer loan,Installment loan,Managing the loan or lease,Unknown or not specified,OH,44241.0,2015-03-18,2015-03-18,"CashCall, Inc.",Closed with explanation,Yes,Yes
9,1288304,Debt collection,Payday loan,Taking/threatening an illegal action,Threatened arrest/jail if do not pay,TX,77433.0,2015-03-18,2015-03-18,Cottonwood Financial Ltd.,Closed with explanation,Yes,Yes
14,1289900,Debt collection,"Other (phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,UT,84405.0,2015-03-18,2015-03-18,Account Liquidation Services Inc.,Closed with non-monetary relief,Yes,Yes
22,1289844,Debt collection,Unknown or not specified,Improper contact or sharing of info,Contacted me after I asked not to,IL,60411.0,2015-03-18,2015-03-18,"Municipal Collections Services, Inc.",Closed with explanation,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28149,1178177,Credit reporting,Unknown or not specified,Improper use of my credit report,Report improperly shared by CRC,MI,48622.0,2015-01-01,2015-01-07,Experian,Closed with explanation,Yes,No
28150,1178179,Credit card,Unknown or not specified,Late fee,Unknown or not specified,AR,72160.0,2015-01-01,2015-01-02,U.S. Bancorp,Closed with monetary relief,Yes,No
28151,1178184,Debt collection,Unknown or not specified,Cont'd attempts collect debt not owed,Debt is not mine,TX,75201.0,2015-01-01,2015-01-02,"Enhanced Recovery Company, LLC",Closed with non-monetary relief,Yes,No
28153,1178182,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",Unknown or not specified,MN,55105.0,2015-01-01,2015-01-02,Regions,Closed with non-monetary relief,Yes,No


In [5]:
df=df[df["Company response"]!="In progress"]
df

,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",Unknown or not specified,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
6,1288848,Consumer loan,Installment loan,Managing the loan or lease,Unknown or not specified,OH,44241.0,2015-03-18,2015-03-18,"CashCall, Inc.",Closed with explanation,Yes,Yes
9,1288304,Debt collection,Payday loan,Taking/threatening an illegal action,Threatened arrest/jail if do not pay,TX,77433.0,2015-03-18,2015-03-18,Cottonwood Financial Ltd.,Closed with explanation,Yes,Yes
14,1289900,Debt collection,"Other (phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,UT,84405.0,2015-03-18,2015-03-18,Account Liquidation Services Inc.,Closed with non-monetary relief,Yes,Yes
22,1289844,Debt collection,Unknown or not specified,Improper contact or sharing of info,Contacted me after I asked not to,IL,60411.0,2015-03-18,2015-03-18,"Municipal Collections Services, Inc.",Closed with explanation,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28149,1178177,Credit reporting,Unknown or not specified,Improper use of my credit report,Report improperly shared by CRC,MI,48622.0,2015-01-01,2015-01-07,Experian,Closed with explanation,Yes,No
28150,1178179,Credit card,Unknown or not specified,Late fee,Unknown or not specified,AR,72160.0,2015-01-01,2015-01-02,U.S. Bancorp,Closed with monetary relief,Yes,No
28151,1178184,Debt collection,Unknown or not specified,Cont'd attempts collect debt not owed,Debt is not mine,TX,75201.0,2015-01-01,2015-01-02,"Enhanced Recovery Company, LLC",Closed with non-monetary relief,Yes,No
28153,1178182,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",Unknown or not specified,MN,55105.0,2015-01-01,2015-01-02,Regions,Closed with non-monetary relief,Yes,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6006 entries, 2 to 28155
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Complaint ID          6006 non-null   int64 
 1   Product               6006 non-null   object
 2   Sub-product           6006 non-null   object
 3   Issue                 6006 non-null   object
 4   Sub-issue             6006 non-null   object
 5   State                 6006 non-null   object
 6   ZIP code              6006 non-null   object
 7   Date received         6006 non-null   object
 8   Date sent to company  6006 non-null   object
 9   Company               6006 non-null   object
 10  Company response      6006 non-null   object
 11  Timely response?      6006 non-null   object
 12  Consumer disputed?    6006 non-null   object
dtypes: int64(1), object(12)
memory usage: 656.9+ KB


In [7]:
pipe=tabular_learner(estimator="classifier")

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\skrub\_tabular_pipeline.py:75: FutureWarning: tabular_learner will be deprecated in the next release. Equivalent functionality is available in skrub.tabular_pipeline.
  warnings.warn(


In [8]:
pipe

Pipeline(steps=[('tablevectorizer',
                 TableVectorizer(low_cardinality=ToCategorical())),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [9]:
X=df.drop(columns=["Consumer disputed?", "Complaint ID" , "ZIP code",   "Date received", "Date sent to company" ])
y=df["Consumer disputed?"]

In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6006 entries, 2 to 28155
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product           6006 non-null   object
 1   Sub-product       6006 non-null   object
 2   Issue             6006 non-null   object
 3   Sub-issue         6006 non-null   object
 4   State             6006 non-null   object
 5   Company           6006 non-null   object
 6   Company response  6006 non-null   object
 7   Timely response?  6006 non-null   object
dtypes: object(8)
memory usage: 422.3+ KB


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, train_size=0.2, random_state=True, shuffle=True)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Identify feature types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()

# Numeric preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing with type conversion fix
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('to_str', FunctionTransformer(lambda x: x.astype(str))),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Complete pipeline with ADASYN
model_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('oversampler', SMOTE(random_state=42)),
    ('classifier', KNeighborsClassifier())
])

# Train the pipeline
model_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('to_str',
                                                                   FunctionTransformer(func=<function <lambda> at 0x00000233564118A0>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Product', 'Sub-product',
                                                   'Issue', 'Sub-issue',
                                                   'State', 'Company',
                                                   'Company response',
                                                   'Timely response?'])])),
                ('oversampler', SMOTE(random_state=42)),
                ('classifier', KNeighborsClassifier())])

In [16]:

y_probs = model_pipeline.predict_proba(X_test)[:, 1]
y_pred = pd.DataFrame((y_probs > 0.85).astype(int))  
y_pred

,0
0,0
1,0
2,0
3,0
4,0
...,...
4800,0
4801,0
4802,0
4803,0


In [17]:
y_test=pd.DataFrame(y_test).replace(to_replace=["Yes", "No"], value=[1, 0] )
y_test

C:\Users\unaif\AppData\Local\Temp\ipykernel_14060\3279481526.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_test=pd.DataFrame(y_test).replace(to_replace=["Yes", "No"], value=[1, 0] )


,Consumer disputed?
17057,1
28052,0
15542,1
10216,1
22213,1
...,...
27941,0
18185,1
11216,1
24934,0


In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred, normalize='true'))

Accuracy: 0.28283038501560875
Classification Report:
              precision    recall  f1-score   support

           0       0.22      0.94      0.36      1033
           1       0.86      0.10      0.18      3772

    accuracy                           0.28      4805
   macro avg       0.54      0.52      0.27      4805
weighted avg       0.72      0.28      0.22      4805

Confusion Matrix:
[[0.93707648 0.06292352]
 [0.89634146 0.10365854]]


In [20]:
from sklearn.model_selection import cross_validate
cross_val=cross_validate(model_pipeline, X_train, y_train, cv=10)["test_score"]

print(cross_val.mean())
print(cross_val.std())

0.41630165289256205
0.0426178572915823


In [ ]:
---

In [ ]:
import os 
import pickle
os.chdir("../models")
import pickle

# Cargar el archivo .pkl
with open('modelo_timely_tree.pkl', 'rb') as f:
    objeto = pickle.load(f)